In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [2]:
movie=pd.read_csv('tmdb_5000_movies.csv')
credit=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
credit['id']=credit['movie_id']
credit.drop('movie_id', axis=1, inplace=True)

In [6]:
data=movie.merge(credit, on='id')

In [7]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
data.drop(['homepage', 'title_x', 'title_y', 'status', 'production_countries'], axis=1, inplace=True)

In [9]:
data['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [10]:
data['overview'].fillna('', inplace=True)

In [11]:
TF=TfidfVectorizer(stop_words='english', analyzer='word', min_df=3, strip_accents='unicode', ngram_range=(1,3), token_pattern=r'\w{1,}', max_features=None)

In [12]:
vec=TF.fit_transform(data['overview'])

In [13]:
vec

<4803x10417 sparse matrix of type '<class 'numpy.float64'>'
	with 127220 stored elements in Compressed Sparse Row format>

In [14]:
sig=sigmoid_kernel(vec,vec)

In [15]:
sig

array([[0.76163447, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76163447, 0.76159416, ..., 0.76159513, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76163447, ..., 0.76159486, 0.76159416,
        0.76159455],
       ...,
       [0.76159416, 0.76159513, 0.76159486, ..., 0.76163447, 0.76159483,
        0.76159473],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159483, 0.76163447,
        0.76159461],
       [0.76159416, 0.76159416, 0.76159455, ..., 0.76159473, 0.76159461,
        0.76163447]])

In [16]:
index=pd.Series(data.index, index=data['original_title'])

In [17]:
index

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [18]:
index['Shanghai Calling']

4801

In [19]:
sig[4801]

array([0.76159416, 0.76159416, 0.76159416, ..., 0.76159483, 0.76163447,
       0.76159461])

In [20]:
list(enumerate(sig[index['Shanghai Calling']]))

[(0, 0.7615941559557649),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.7615957266283515),
 (4, 0.7615941559557649),
 (5, 0.7615943593980157),
 (6, 0.7615945340984431),
 (7, 0.7615941559557649),
 (8, 0.7615953936982669),
 (9, 0.761594367248053),
 (10, 0.7615941559557649),
 (11, 0.7615941559557649),
 (12, 0.7615941559557649),
 (13, 0.7615941559557649),
 (14, 0.7615955938506791),
 (15, 0.7615947495664399),
 (16, 0.7615941559557649),
 (17, 0.7615941559557649),
 (18, 0.7615944439688758),
 (19, 0.7615950630300181),
 (20, 0.7615941559557649),
 (21, 0.7615941559557649),
 (22, 0.7615941559557649),
 (23, 0.7615941559557649),
 (24, 0.7615980254629633),
 (25, 0.7615943885039911),
 (26, 0.7615941559557649),
 (27, 0.7615941559557649),
 (28, 0.7615941559557649),
 (29, 0.7615950590140795),
 (30, 0.7615941559557649),
 (31, 0.7615941559557649),
 (32, 0.7615944342377062),
 (33, 0.7615941559557649),
 (34, 0.7615941559557649),
 (35, 0.7615965027106367),
 (36, 0.7615948927771239),
 (37, 0.7615

In [21]:
def recommend(title, sig=sig):
    place=index[title]
    sig_score=list(enumerate(sig[place]))
    sig_score=sorted(sig_score, key=lambda x:x[1], reverse=True)
    sig_score=sig_score[1:21]
    indices=[i[0] for i in sig_score]
    return data['original_title'].iloc[indices]

In [22]:
recommend("Amour")

3723                           Anne of Green Gables
3146                                Poltergeist III
1602                                     Blue Crush
3496                                La fleur du mal
3513                                       Song One
4799                                      Newlyweds
1240                          The Other Boleyn Girl
2238                                       Bandslam
2402                                     The Others
2960                                            LOL
2689                             Our Family Wedding
3945                             Love's Abiding Joy
1396                                    Poltergeist
17      Pirates of the Caribbean: On Stranger Tides
2400                                     The Prince
1032                          America's Sweethearts
4207                                Out of the Dark
1511              Dreamer: Inspired By a True Story
3610                              Not Easily Broken
2764        

In [23]:
#Overview of "Anne of Green Gables"
data.at[3723,'overview']

"At the turn of the century on Prince Edward Island, Matthew Cuthbert and his sister Marilla decide to take on an orphan boy as help for their farm. But they get an unexpected jolt when they're mistakenly sent a girl instead: Anne Shirley. Anne's a dreamer with an unusual point of view, far removed from Marilla's pragmatic ways, and it's only on trial that Marilla agrees to keep Anne."

In [24]:
#Overview of "Amour"
data.at[3145,'overview'] 

"Georges and Anne are in their eighties. They are cultivated, retired music teachers. Their daughter, who is also a musician, lives abroad with her family. One day, Anne has a stroke, and the couple's bond of love is severely tested."

In [25]:
#Overview of "Poltergeist III"
data.at[3146,'overview']

"Carol Anne has been sent to live with her Aunt and Uncle in an effort to hide her from the clutches of the ghostly Reverend Kane, but he tracks her down and terrorises her in her relatives' appartment in a tall glass building. Will he finally achieve his target and capture Carol Anne again, or will Tangina be able, yet again, to thwart him?"

In [26]:
#Overview of "Blue Crush"
data.at[1602,'overview']

"Nothing gets between Anne Marie and her board. Living in a beach shack with three roommates, she is up before dawn every morning to conquer the waves and count the days until the Pipe Masters competition. Having transplanted herself to Hawaii with no one's blessing but her own, Anne Marie finds all she needs in the adrenaline-charged surf scene - until pro quarterback Matt Tollman comes along..."

In [27]:
recommend("Avatar")

1341                   Obitaemyy Ostrov
634                          The Matrix
3604                          Apollo 18
2130                       The American
775                           Supernova
529                    Tears of the Sun
151                             Beowulf
311        The Adventures of Pluto Nash
847                            Semi-Pro
942                    The Book of Life
36      Transformers: Age of Extinction
570                              Ransom
1610                              Hanna
3070            Jeff, Who Lives at Home
1784                          EverAfter
1033                           Insomnia
2578                         The Marine
1013                           Child 44
3724                      Falcon Rising
4211            La sirène du Mississipi
Name: original_title, dtype: object

In [28]:
#Overview of "Avatar"
data.at[0,'overview']

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [29]:
#Overview of "Obitaemyy Ostrov"
data.at[1341,'overview']

'On the threshold of 22nd century, furrowing the space, protagonist from the Free Search Group makes emergency landing on an unknown planet where he must stay. People who are living on this planet have remained at the stone level of the 20th century, with its social problems, miserable ecology and shaky world..'

In [30]:
#Overview of "The Matrix"
data.at[634,'overview']

'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.'

In [31]:
recommend("Titanic")

1269                                  Raise the Titanic
2289                                         The Switch
296                                         End of Days
2287                         I Can Do Bad All By Myself
2143                                         Ghost Ship
4287                                            Niagara
171     Master and Commander: The Far Side of the World
4035                                              Stung
4228                        The Ballad of Jack and Rose
104                                            Poseidon
3212                                           The Rose
770                                       Event Horizon
2955                               Romance & Cigarettes
2902                                           Triangle
1294                                           Serenity
2129                                     The Black Hole
775                                           Supernova
3103                     La femme de chambre du 

In [32]:
data.at[1269,'overview']

'To obtain a supply of a rare mineral, a ship raising operation is conducted for the only known source, the Titanic.'

In [33]:
data.at[2289,'overview']

'An unmarried 40-year-old woman turns to a turkey baster in order to become pregnant. Seven years later, she reunites with her best friend, who has been living with a secret: he replaced her preferred sperm sample with his own.'